In [1]:
import os
import pandas as pd
import pickle
import json
import requests
import ast

import turicreate
from turicreate import SFrame

import azureml.core
from azureml.core import Workspace
from azureml.core.model import Model, InferenceConfig
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.webservice import AciWebservice, LocalWebservice, Webservice,  AksWebservice
from azureml.core.compute import AksCompute, ComputeTarget
from azureml.core.compute_target import ComputeTargetException

In [2]:
print("AZML SDK Version:", azureml.core.VERSION)
print("Pandas Version:", pd.__version__)

AZML SDK Version: 1.43.0
Pandas Version: 1.1.5


In [3]:
# If in Databricks
# OUTPUT_DIR = "/dbfs/FileStore/RetailRecommender/output/"
# MODELS_DIR = "/dbfs/FileStore/RetailRecommender/models/"
# DEPLOY_DIR = "/dbfs/FileStore/RetailRecommender/deploy/"

OUTPUT_DIR = "./output/"
MODELS_DIR = "./models/"
DEPLOY_DIR = "./deploy/"

os.makedirs(DEPLOY_DIR, exist_ok=True)

# 1. Load the AML Workspace

In [4]:
ws = Workspace(workspace_name="YOUR_WORKSPACE_NAME", 
                subscription_id="YOUR_SUBSCRIPTION_ID", 
                resource_group="YOUR_RESOURCE_GROUP")
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = "\n")

pabmar-workspace
pabmar-rg
southcentralus
b1395605-1fe9-4af4-b3ff-82a4725a3791


# 2. Register the Models on AZML service model registry

In [5]:
model_s_path = MODELS_DIR + "recommendation_s.model"
model_r_path = MODELS_DIR + "recommendation_r.model"

In [6]:
azmodel_s = Model.register( model_path = model_s_path,
                           model_name = "similarity_model",
                           workspace = ws)

azmodel_r = Model.register( model_path = model_r_path,
                           model_name = "ranking_factorization_model",
                           workspace = ws)

Registering model similarity_model
Registering model ranking_factorization_model


# 3. Test score.py init() and run() functions

In [11]:
import os

def init():
    
    global model_s
    global model_r
    global item_column_name
    global user_column_name
    global rating_column_name
    
    item_column_name = "itemid"
    user_column_name = "visitorid"
    rating_column_name = "event"
    
    model_s = turicreate.load_model(model_s_path)
    model_r = turicreate.load_model(model_r_path)
    

In [14]:
def run(raw_data):
    
    data = json.loads(raw_data)
    data = pd.DataFrame(data).to_dict(orient="list")

    try:
        if user_column_name in data:
            print(user_column_name + " found")
            
            # If observation side data exists
            if ("period" or "month" or "weekday") in data:
                print("Using Ranking Factorization Model\n")
                users_query = turicreate.SFrame(
                    {"visitorid": data[user_column_name], 
                    "period": data["period"] if "period" in data else ["Night"] * len(data[user_column_name]),
                    "month": data["month"] if "month" in data else [8] * len(data[user_column_name]),
                    "weekday": data["weekday"] if "weekday" in data else [2] * len(data[user_column_name])
                    }
                )
                result = model_r.recommend(users=users_query, k=10).to_dataframe()
                
            else:
                print("Using Similarity Model\n")
                result = model_s.recommend(users=data[user_column_name], k=10).to_dataframe()
                
            if item_column_name in data:
                return "Error: Request cannot have" + item_column_name + " and" + user_column_name + " together. Please make separate requests for user recommendations and for items basket similarity"
        
        elif item_column_name in data:
                print("Using Similarity Model\n")
                result = model_s.get_similar_items(data[item_column_name], k=10).to_dataframe()
                
        else:
            return "Error: Payload must contain at least a field named " + user_column_name + " or " + item_column_name
    
        return result.to_json(orient="records")
        
    except Exception as e:
        return e


In [15]:
init()

In [28]:

test_load1 = '''
    [
        {"visitorid":6000, "period":"Night",  "weekday":2},
        {"visitorid":567987987988, "period":"Night", "weekday":2}
    ]
'''

test_load2 = '''
    [
        {"visitorid":6000},
        {"visitorid":567987987988}
    ]
'''

test_load3 = '''
    [
        {"visitorid":6000,"period":"Night",  "weekday":2 , "month": 9},
        {"visitorid":567987987988,"period":"Night",  "weekday":2 , "month": 9}
    ]
'''

test_load4 = '''
    [
        {"itemid":152913},
        {"itemid":355908}
    ]
'''


In [24]:
run(test_load3)

visitorid found
Using Ranking Factorization Model



'[{"visitorid":6000,"itemid":205111,"score":1.2024936294,"rank":1},{"visitorid":6000,"itemid":238804,"score":1.2009366208,"rank":2},{"visitorid":6000,"itemid":257723,"score":1.2006300153,"rank":3},{"visitorid":6000,"itemid":379272,"score":1.1984604588,"rank":4},{"visitorid":6000,"itemid":149723,"score":1.1954857732,"rank":5},{"visitorid":6000,"itemid":217003,"score":1.1951026849,"rank":6},{"visitorid":6000,"itemid":259608,"score":1.1949490726,"rank":7},{"visitorid":6000,"itemid":184079,"score":1.1944965143,"rank":8},{"visitorid":6000,"itemid":407706,"score":1.1941066739,"rank":9},{"visitorid":6000,"itemid":32164,"score":1.1931563363,"rank":10},{"visitorid":567987987988,"itemid":205111,"score":1.1218765325,"rank":1},{"visitorid":567987987988,"itemid":257723,"score":1.119803068,"rank":2},{"visitorid":567987987988,"itemid":238804,"score":1.1183074689,"rank":3},{"visitorid":567987987988,"itemid":379272,"score":1.1177684607,"rank":4},{"visitorid":567987987988,"itemid":407706,"score":1.11553

In [29]:
run(test_load4)

Using Similarity Model



'[{"itemid":152913,"similar":305216,"score":0.125,"rank":1},{"itemid":152913,"similar":139923,"score":0.125,"rank":2},{"itemid":152913,"similar":393137,"score":0.0833333135,"rank":3},{"itemid":152913,"similar":311292,"score":0.0476190448,"rank":4},{"itemid":152913,"similar":409791,"score":0.0434782505,"rank":5},{"itemid":152913,"similar":211299,"score":0.0204081535,"rank":6},{"itemid":152913,"similar":155485,"score":0.0117647052,"rank":7},{"itemid":355908,"similar":97139,"score":0.0858895779,"rank":1},{"itemid":355908,"similar":238777,"score":0.0756302476,"rank":2},{"itemid":355908,"similar":259725,"score":0.0731707215,"rank":3},{"itemid":355908,"similar":436412,"score":0.0722891688,"rank":4},{"itemid":355908,"similar":338208,"score":0.0676691532,"rank":5},{"itemid":355908,"similar":393910,"score":0.0634920597,"rank":6},{"itemid":355908,"similar":153118,"score":0.0634920597,"rank":7},{"itemid":355908,"similar":298868,"score":0.0588235259,"rank":8},{"itemid":355908,"similar":419387,"sco

# 4. Create Inference Environment

In [31]:
# curated_env = Environment.get(workspace=ws, name="AzureML-minimal-ubuntu18.04-py37-cpu-inference")
# myenv = curated_env.clone("turicreate_env")

conda_dep = CondaDependencies()
conda_dep.add_pip_package("turicreate")

myenv = Environment(name="turicreate_env")
myenv.python.conda_dependencies=conda_dep
myenv.inferencing_stack_version = "latest"

# Register the Environment on AZML so we can retreive it later
myenv = myenv.register(workspace=ws)

# Uncomment this to re-build the environment
# myenv.build(workspace=ws)


Environment version is set. Attempting to register desired version. To auto-version, reset version to None.


# 5. Create the inference configuration

### !!! Make sure that in score.py you set the right model version!!!

In [49]:
scorepy_text = '''
import os
import sys
import turicreate
from turicreate import SFrame
import pandas as pd
import json

def init():
    
    global model_s
    global model_r
    
    model_s_name = "similarity_model"
    model_s_version = "{S_VERSION}"
    model_s_filename = "recommendation_s.model"
    
    model_r_name = "ranking_factorization_model"
    model_r_version = "{R_VERSION}"  
    model_r_filename = "recommendation_r.model"
    
    model_s_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), model_s_name, model_s_version, model_s_filename)
    model_r_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), model_r_name, model_r_version, model_r_filename)
    
    print("model_s_path",model_s_path)
    print("model_r_path",model_r_path)
    
    model_s = turicreate.load_model(model_s_path)
    model_r = turicreate.load_model(model_r_path)
    
    
def run(raw_data):
    
    data = json.loads(raw_data)
    data = pd.DataFrame(data).to_dict(orient="list")

    try:
        if "{USER_ID}" in data:
            print("{USER_ID} found")
            
            # If observation side data exists
            if ("period" or "month" or "weekday") in data:
                print("Using Ranking Factorization Model")
                users_query = turicreate.SFrame(
                    {{"visitorid": data["{USER_ID}"], 
                    "period": data["period"] if "period" in data else ["Night"] * len(data["{USER_ID}"]),
                    "month": data["month"] if "month" in data else [8] * len(data["{USER_ID}"]),
                    "weekday": data["weekday"] if "weekday" in data else [2] * len(data["{USER_ID}"])
                    }}
                )
                result = model_r.recommend(users=users_query, k=10).to_dataframe()
                
            else:
                print("Using Similarity Model")
                result = model_s.recommend(users=data["{USER_ID}"], k=10).to_dataframe()
                
            if "{ITEM_ID}" in data:
                return "Error: Request cannot have {USER_ID} and {ITEM_ID} together. Please make separate requests for {USER_ID} recommendations and for {ITEM_ID} basket similarity"
        
        elif "{ITEM_ID}" in data:
                print("Using Similarity Model")
                result = model_s.get_similar_items(data["{ITEM_ID}"], k=10).to_dataframe()
                
        else:
            return "Error: Payload must contain at least a field named {USER_ID} or {ITEM_ID}"
    
        return result.to_json(orient="records")
        
    except Exception as e:
        return e

'''.format(S_VERSION=6, R_VERSION=1, USER_ID=user_column_name, ITEM_ID=item_column_name)

with open(DEPLOY_DIR+"score.py", "w") as file:
    file.write(scorepy_text)

print("score.py written, using",DEPLOY_DIR+"score.py")

score.py written, using ./deploy/score.py


In [50]:
!cat {DEPLOY_DIR}score.py


import os
import sys
import turicreate
from turicreate import SFrame
import pandas as pd
import json

def init():
    
    global model_s
    global model_r
    
    model_s_name = "similarity_model"
    model_s_version = "6"
    model_s_filename = "recommendation_s.model"
    
    model_r_name = "ranking_factorization_model"
    model_r_version = "1"  
    model_r_filename = "recommendation_r.model"
    
    model_s_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), model_s_name, model_s_version, model_s_filename)
    model_r_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), model_r_name, model_r_version, model_r_filename)
    
    print("model_s_path",model_s_path)
    print("model_r_path",model_r_path)
    
    model_s = turicreate.load_model(model_s_path)
    model_r = turicreate.load_model(model_r_path)
    
    
def run(raw_data):
    
    data = json.loads(raw_data)
    data = pd.DataFrame(data).to_dict(orient="list")

    try:
        if "visitorid" in data:
            pri

# 6 . Deploy to Local
This does not work on Databricks

In [51]:
inference_config = InferenceConfig(entry_script=DEPLOY_DIR+"score.py",environment=myenv)

deployment_config = LocalWebservice.deploy_configuration(port=6789)

local_service = Model.deploy(ws, "test", [azmodel_s, azmodel_r], inference_config, deployment_config)

Generating Docker build context.
Package creation Succeeded
Logging into Docker registry 063bc338e140482aa343d08429014bdd.azurecr.io
Logging into Docker registry 063bc338e140482aa343d08429014bdd.azurecr.io
Building Docker image from Dockerfile...
Step 1/5 : FROM 063bc338e140482aa343d08429014bdd.azurecr.io/azureml/azureml_b3ed4f560aaa538365ef8fe691b671c6
 ---> dc119dbce441
Step 2/5 : COPY azureml-app /var/azureml-app
 ---> 0254afe1dd1c
Step 3/5 : RUN mkdir -p '/var/azureml-app' && echo eyJhY2NvdW50Q29udGV4dCI6eyJzdWJzY3JpcHRpb25JZCI6ImIxMzk1NjA1LTFmZTktNGFmNC1iM2ZmLTgyYTQ3MjVhMzc5MSIsInJlc291cmNlR3JvdXBOYW1lIjoicGFibWFyLXJnIiwiYWNjb3VudE5hbWUiOiJwYWJtYXItd29ya3NwYWNlIiwid29ya3NwYWNlSWQiOiIwNjNiYzMzOC1lMTQwLTQ4MmEtYTM0My1kMDg0MjkwMTRiZGQifSwibW9kZWxzIjp7fSwibW9kZWxzSW5mbyI6e319 | base64 --decode > /var/azureml-app/model_config_map.json
 ---> Running in ee2e1e8c9607
 ---> eb3514092193
Step 4/5 : RUN mv '/var/azureml-app/tmp6k7s6d8c.py' /var/azureml-app/main.py
 ---> Running in 56dfbd239e4

In [52]:
# If container fails, check out the logs
# local_service.get_logs()

## Test local web service container

In [53]:
print("Local service hosted in:", local_service.scoring_uri)

Local service hosted in: http://localhost:6789/score


In [54]:
import json

response = local_service.run(input_data = test_load3)
print(response)

Checking container health...
Local webservice is running at http://localhost:6789
[{"visitorid":6000,"itemid":205111,"score":1.2024936294,"rank":1},{"visitorid":6000,"itemid":238804,"score":1.2009366208,"rank":2},{"visitorid":6000,"itemid":257723,"score":1.2006300153,"rank":3},{"visitorid":6000,"itemid":379272,"score":1.1984604588,"rank":4},{"visitorid":6000,"itemid":149723,"score":1.1954857732,"rank":5},{"visitorid":6000,"itemid":217003,"score":1.1951026849,"rank":6},{"visitorid":6000,"itemid":259608,"score":1.1949490726,"rank":7},{"visitorid":6000,"itemid":184079,"score":1.1944965143,"rank":8},{"visitorid":6000,"itemid":407706,"score":1.1941066739,"rank":9},{"visitorid":6000,"itemid":32164,"score":1.1931563363,"rank":10},{"visitorid":567987987988,"itemid":205111,"score":1.1218765325,"rank":1},{"visitorid":567987987988,"itemid":257723,"score":1.119803068,"rank":2},{"visitorid":567987987988,"itemid":238804,"score":1.1183074689,"rank":3},{"visitorid":567987987988,"itemid":379272,"score"

In [55]:
import urllib
import requests
import ast

# The URL will need to the edited if ACI or AKS service
url_aci = local_service.scoring_uri

headers = {'Content-Type':'application/json'}
body = test_load2

req = urllib.request.Request(url_aci, str.encode(body), headers)
response = ast.literal_eval(urllib.request.urlopen(req).read().decode('utf-8'))

print(response)


[{"visitorid":6000,"itemid":64700,"score":0.0598958333,"rank":1},{"visitorid":6000,"itemid":158457,"score":0.0555555622,"rank":2},{"visitorid":6000,"itemid":54572,"score":0.0476190448,"rank":3},{"visitorid":6000,"itemid":342866,"score":0.0476190448,"rank":4},{"visitorid":6000,"itemid":393111,"score":0.0476190448,"rank":5},{"visitorid":6000,"itemid":464114,"score":0.0476190448,"rank":6},{"visitorid":6000,"itemid":50575,"score":0.0476190448,"rank":7},{"visitorid":6000,"itemid":279324,"score":0.0416666667,"rank":8},{"visitorid":6000,"itemid":342139,"score":0.0416666667,"rank":9},{"visitorid":6000,"itemid":26634,"score":0.0388888915,"rank":10},{"visitorid":567987987988,"itemid":456056,"score":0.0257491243,"rank":1},{"visitorid":567987987988,"itemid":119736,"score":0.0251864088,"rank":2},{"visitorid":567987987988,"itemid":338395,"score":0.0233529449,"rank":3},{"visitorid":567987987988,"itemid":439963,"score":0.0216234672,"rank":4},{"visitorid":567987987988,"itemid":186702,"score":0.02007171

# 7. Deploy to Azure Container Instance

In [56]:
inference_config = InferenceConfig(entry_script=DEPLOY_DIR+"score.py",environment=myenv)

deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

aci_service = Model.deploy(ws, "aci-service", [azmodel_s, azmodel_r], inference_config, deployment_config, overwrite=True)

aci_service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-12-07 00:04:35+00:00 Creating Container Registry if not exists.
2022-12-07 00:04:35+00:00 Registering the environment.
2022-12-07 00:04:37+00:00 Use the existing image.
2022-12-07 00:04:37+00:00 Generating deployment configuration.
2022-12-07 00:04:40+00:00 Submitting deployment to compute.
2022-12-07 00:04:43+00:00 Checking the status of deployment test..
2022-12-07 00:09:51+00:00 Checking the status of inference endpoint test.
Succeeded
ACI service creation operation finished, operation "Succeeded"


## Test ACI URL API Endpoint

In [58]:
aci_service.scoring_uri

'http://71f08952-e588-491d-a93e-40ce53b4cff3.southcentralus.azurecontainer.io/score'

In [57]:
import urllib
import requests
import ast

# The URL will need to the edited if ACI or AKS service
url_aci = aci_service.scoring_uri

headers = {'Content-Type':'application/json'}
body = test_load2

req = urllib.request.Request(url_aci, str.encode(body), headers)
response = ast.literal_eval(urllib.request.urlopen(req).read().decode('utf-8'))

print(response)


[{"visitorid":6000,"itemid":64700,"score":0.0598958333,"rank":1},{"visitorid":6000,"itemid":158457,"score":0.0555555622,"rank":2},{"visitorid":6000,"itemid":54572,"score":0.0476190448,"rank":3},{"visitorid":6000,"itemid":342866,"score":0.0476190448,"rank":4},{"visitorid":6000,"itemid":393111,"score":0.0476190448,"rank":5},{"visitorid":6000,"itemid":464114,"score":0.0476190448,"rank":6},{"visitorid":6000,"itemid":50575,"score":0.0476190448,"rank":7},{"visitorid":6000,"itemid":279324,"score":0.0416666667,"rank":8},{"visitorid":6000,"itemid":342139,"score":0.0416666667,"rank":9},{"visitorid":6000,"itemid":26634,"score":0.0388888915,"rank":10},{"visitorid":567987987988,"itemid":456056,"score":0.0257491243,"rank":1},{"visitorid":567987987988,"itemid":119736,"score":0.0251864088,"rank":2},{"visitorid":567987987988,"itemid":338395,"score":0.0233529449,"rank":3},{"visitorid":567987987988,"itemid":439963,"score":0.0216234672,"rank":4},{"visitorid":567987987988,"itemid":186702,"score":0.02007171

In [59]:
aci_service.delete()

# 8. Deploy to Production -  managed AKS Cluster

Details for SSL and more best practices for production here: https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/deployment/production-deploy-to-aks/production-deploy-to-aks.ipynb

In [60]:
# Choose a name for your AKS cluster
aks_name = 'my-aks'

# Verify that cluster does not exist already
try:
    aks_target = ComputeTarget(workspace=ws, name=aks_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # Use the default configuration (can also provide parameters to customize)
    # https://learn.microsoft.com/en-us/python/api/azureml-core/azureml.core.compute.aks.akscompute?view=azure-ml-py
    prov_config = AksCompute.provisioning_configuration()

    # Create the cluster
    aks_target = ComputeTarget.create(workspace = ws, 
                                    name = aks_name, 
                                    provisioning_configuration = prov_config)

if aks_target.get_status() != "Succeeded":
    aks_target.wait_for_completion(show_output=True)

Found existing cluster, use it.


In [63]:
# Set the web service configuration (using default here)
# https://learn.microsoft.com/en-us/python/api/azureml-core/azureml.core.webservice.akswebservice?view=azure-ml-py#azureml-core-webservice-akswebservice-deploy-configuration
aks_config = AksWebservice.deploy_configuration()

# # Enable token auth and disable (key) auth on the webservice
# aks_config = AksWebservice.deploy_configuration(token_auth_enabled=True, auth_enabled=False)

In [64]:
%%time

aks_service_name ='aks-service'

aks_service = Model.deploy(workspace=ws,
                           name=aks_service_name,
                           models=[azmodel_s, azmodel_r],
                           inference_config=inference_config,
                           deployment_config=aks_config,
                           deployment_target=aks_target,
                           overwrite=True)

aks_service.wait_for_deployment(show_output = True)
print(aks_service.state)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-12-07 00:11:49+00:00 Creating Container Registry if not exists.
2022-12-07 00:11:49+00:00 Registering the environment.
2022-12-07 00:11:50+00:00 Use the existing image.
2022-12-07 00:11:54+00:00 Checking the status of deployment aks-service..
2022-12-07 00:12:25+00:00 Checking the status of inference endpoint aks-service.
Succeeded
AKS service creation operation finished, operation "Succeeded"
Healthy
CPU times: user 373 ms, sys: 86.9 ms, total: 460 ms
Wall time: 42.9 s


# 8. Test the web service using raw HTTP request

In [65]:
# # if (key) auth is enabled, retrieve the API keys. AML generates two keys.
key1, Key2 = aks_service.get_keys()
print(key1)

# # if token auth is enabled, retrieve the token.
# access_token, refresh_after = aks_service.get_token()

G46wSFtiiAFlCHHi0beyU29ceaB2dPRS


In [66]:
print(aks_service.scoring_uri)

http://20.114.104.171:80/api/v1/service/aks-service/score


In [68]:
apiurl = aks_service.scoring_uri
key = key1
# key = "G46wSFtiiAFlCHHi0beyU29ceaB2dPRS"
# apiurl = "http://20.114.104.171:80/api/v1/service/aks-service/score"

In [69]:
%%time

# # If (key) auth is enabled, don't forget to add key to the HTTP header.
headers = {'Content-Type':'application/json', 'Authorization': 'Bearer ' + key}

# # If token auth is enabled, don't forget to add token to the HTTP header.
# headers = {'Content-Type':'application/json', 'Authorization': 'Bearer ' + access_token}

resp = requests.post(apiurl, bytes(test_load1,encoding = 'utf8'), headers=headers)

print(ast.literal_eval(resp.text))

[{"visitorid":6000,"itemid":119736,"score":1.2709273953,"rank":1},{"visitorid":6000,"itemid":32227,"score":1.2562527951,"rank":2},{"visitorid":6000,"itemid":234215,"score":1.2536956422,"rank":3},{"visitorid":6000,"itemid":210002,"score":1.2502753945,"rank":4},{"visitorid":6000,"itemid":23762,"score":1.2486156024,"rank":5},{"visitorid":6000,"itemid":76196,"score":1.2428713669,"rank":6},{"visitorid":6000,"itemid":212006,"score":1.2396474962,"rank":7},{"visitorid":6000,"itemid":256721,"score":1.2389497782,"rank":8},{"visitorid":6000,"itemid":123664,"score":1.2384240537,"rank":9},{"visitorid":6000,"itemid":317192,"score":1.2381217134,"rank":10},{"visitorid":567987987988,"itemid":119736,"score":1.6954194245,"rank":1},{"visitorid":567987987988,"itemid":32227,"score":1.692107864,"rank":2},{"visitorid":567987987988,"itemid":234215,"score":1.6887668235,"rank":3},{"visitorid":567987987988,"itemid":76196,"score":1.6697236534,"rank":4},{"visitorid":567987987988,"itemid":210002,"score":1.669452417,

In [70]:
aks_service.delete()

In [ ]:
# aks_service.get_logs()